In [1]:
import pandas as pd

import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from feature_engine.creation import CyclicalFeatures

from catboost import CatBoostRegressor

from sklearn.model_selection import RandomizedSearchCV

In [23]:
# Load dataframes
path = "../data/train.csv"
df = pd.read_csv(path, parse_dates=[0])

df_weather1 = pd.read_csv("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR/H_75_previous-2020-2022.csv.gz", sep=';')
df_weather2 = pd.read_csv("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR/H_75_latest-2023-2024.csv.gz", sep=';')
weather = pd.concat([df_weather1, df_weather2])

holidays_path = "../data/fr-en-calendrier-scolaire.csv"
holidays = pd.read_csv(holidays_path, sep=";")

traffic_path = "../data/traffic_data.csv"
traffic = pd.read_csv(traffic_path, sep=";")

final_test = pd.read_csv("../data/test.csv", parse_dates=[0])

In [24]:
final_test = pd.read_csv("../data/test.csv", parse_dates=[0]).rename(columns={"id": "date"})

### Data Engineering Class

In [25]:
class Eng:
    def __init__(self, df, weather, traffic, holidays):
        self.df = df
        self.df = self.df.rename(columns={"id": "date"})
        self.df = self.df.drop_duplicates()

        self.external_database(weather, traffic)
        self.additional_data(holidays)
        self.date_features()
        

    def external_database(self, weather, traffic):
        
        ##### Weather #####
        
        self.weather = weather

        # Dropping empty columns
        self.weather = self.weather.drop(
            columns=['FF2', 'QFF2', 'DD2', 'QDD2', 'FXI2', 'QFXI2', 'DXI2', 'QDXI2', 'HXI2', 'QHXI2', 'DXI3S',
                                      'DHUMEC', 'QDHUMEC', 'GEOP', 'QGEOP', 'N', 'QN', 'NBAS', 'QNBAS', 'CL', 'QCL', 'CM', 'QCM',
                                      'CH', 'QCH', 'N1', 'QN1', 'C1', 'QC1', 'B1', 'QB1', 'N2', 'QN2', 'C2', 'QC2', 'B2', 'QB2',
                                      'N3', 'QN3', 'C3', 'QC3', 'B3', 'QB3', 'N4', 'QN4', 'C4', 'QC4', 'B4', 'QB4', 'W1', 'QW1',
                                      'W2', 'QW2', 'SOL', 'QSOL', 'SOLNG', 'QSOLNG', 'TMER', 'QTMER', 'VVMER', 'QVVMER', 'ETATMER',
                                      'QETATMER', 'DIRHOULE', 'QDIRHOULE', 'HVAGUE', 'QHVAGUE', 'PVAGUE', 'QPVAGUE', 'HNEIGEF', 'QHNEIGEF',
                                      'TSNEIGE', 'QTSNEIGE', 'TUBENEIGE', 'QTUBENEIGE', 'HNEIGEFI3', 'QHNEIGEFI3', 'HNEIGEFI1', 'QHNEIGEFI1',
                                      'ESNEIGE', 'QESNEIGE', 'CHARGENEIGE', 'QCHARGENEIGE', 'DIR', 'QDIR', 'DIR2', 'QDIR2', 'DIF', 'QDIF',
                                      'DIF2', 'QDIF2', 'UV', 'QUV', 'UV2', 'QUV2', 'UV_INDICE', 'QUV_INDICE', 'INFRAR', 'QINFRAR', 'INFRAR2',
                                      'QINFRAR2', 'TLAGON', 'QTLAGON', 'TVEGETAUX', 'QTVEGETAUX', 'ECOULEMENT', 'QECOULEMENT', 'NUM_POSTE',
                                      'NOM_USUEL', 'LAT', 'LON', 'DRR1', 'FF', 'QFF', 'DD', 'QDD', 'FXY', 'QFXY', 'DXY', 'QDXY', 'QHXY', 'FXI',
                                      'QFXI', 'DXI', 'QDXI', 'QHXI', 'FXI3S', 'QFXI3S', 'QDXI3S', 'QHFXI3S', 'TD', 'QTD', 'T10', 'QT10', 'T20',
                                      'QT20', 'T50', 'QT50', 'T100', 'QT100', 'TNSOL', 'QTNSOL', 'TN50', 'QTN50', 'TCHAUSSEE', 'QTCHAUSSEE',
                                      'U', 'QU', 'UN', 'QUN', 'QHUN', 'UX', 'QUX', 'QHUX', 'DHUMI40', 'QDHUMI40', 'DHUMI80', 'QDHUMI80', 'TSV',
                                      'QTSV', 'PMER', 'QPMER', 'PSTAT', 'QPSTAT', 'PMERMIN', 'QPMERMIN', 'VV', 'QVV', 'DVV200', 'QDVV200', 'WW',
                                      'QWW', 'NEIGETOT', 'QNEIGETOT', 'GLO', 'QGLO', 'GLO2', 'QGLO2', 'INS', 'QINS', 'INS2', 'QINS2', 'QDRR1']
                                      )
        
        # Grouping by date to sum and average values across all meteo stations
        self.weather["date"] = pd.to_datetime(self.weather["AAAAMMJJHH"], format="%Y%m%d%H")
        self.weather = self.weather.drop(columns=["AAAAMMJJHH", "HXI", "HXY", "HFXI3S", "HTN", "HTX", "HUN", "HUX"])

        self.avg = self.weather.drop(
            columns=['RR1']
            ).groupby("date").mean().reset_index()

        self.tot = self.weather.drop(
            columns=['TX', 'T', 'QTX', 'DG', 'QT', 'QHTN', 'QHTX', 'TN', 'QDG', 'QTN']
            ).groupby("date").sum().reset_index()

        self.weather = self.weather.drop_duplicates(subset=["date"])

        self.df = self.df.merge(self.weather, on="date", how="left")

        # Filling missing values for weather data with their mean as there are very few
        self.is_na = self.df.isna().sum()

        for col in self.is_na.index:
            if self.is_na[col] > 0:
                self.df[col] = self.df[col].fillna(self.df[col].mean())
        

        ##### Traffic #####
        
        # traffic.drop("Unnamed: 0", axis=1, inplace=True)
        traffic["date"] = pd.to_datetime(traffic["date"])

        traffic = traffic.drop_duplicates(subset=["date"])
        linear_int2 = traffic[["date", 'flow', 'occupation_rate']]
        linear_int2 = linear_int2.set_index("date")
        new_features2 = linear_int2.resample('1h').interpolate("linear")

        self.df = self.df.merge(new_features2, on="date", how="left")

        self.df["flow"] = self.df["flow"].fillna(self.df["flow"].mean())
        self.df["occupation_rate"] = self.df["occupation_rate"].fillna(self.df["occupation_rate"].mean())
        

    def date_features(self):
        # Create new date features
        self.df['day'] = self.df['date'].dt.day
        self.df['month'] = self.df['date'].dt.month
        self.df['year'] = self.df['date'].dt.year
        self.df['hour'] = self.df['date'].dt.hour
        self.df['weekday'] = self.df['date'].dt.weekday 
        
        self.df["weekend"] = self.df['weekday'].isin([5, 6])
        

    def additional_data(self, holidays):
    
        ###### Lockdown variable ######
        
        # Lockdown in Paris
        start_date = pd.to_datetime('2020-10-31')
        end_date = pd.to_datetime('2020-12-14')
        start_date1 = pd.to_datetime('2021-04-04')
        end_date1 = pd.to_datetime('2021-05-02')
        
        # Assign 1 if the date is within the specified range, otherwise 0
        self.df['lockdown'] = self.df['date'].apply(lambda x: 1 if ((start_date <= x <= end_date) or (start_date1 <= x <= end_date1)) else 0)
        

        ###### Curfews variable ######
        
        # Curfew date range (+1 hour because the data is the cumulative of the previous)
        curfew_periods = [
            (pd.to_datetime('2020-10-17 22:00'), pd.to_datetime('2020-10-29 07:00')),
            (pd.to_datetime('2020-12-16 21:00'), pd.to_datetime('2021-01-15 07:00')),
            (pd.to_datetime('2021-01-16 19:00'), pd.to_datetime('2021-03-20 07:00')),
            (pd.to_datetime('2021-03-21 20:00'), pd.to_datetime('2021-04-02 07:00')),
            (pd.to_datetime('2021-05-19 22:00'), pd.to_datetime('2021-06-08 07:00')),
            (pd.to_datetime('2021-06-09 23:00'), pd.to_datetime('2021-06-20 07:00'))
        ]

        # Create a new column 'Curfew' and assign 1 if the date is within the specified range, otherwise 0

        self.df['curfew'] = 0  # Initialize the Curfew column

        # Loop through curfew periods and set Curfew column accordingly
        for start_time, end_time in curfew_periods:
            mask = self.df[self.df['date'].between(start_time, end_time)]
            mask = mask[(mask["date"].dt.hour > start_time.hour) | (mask["date"].dt.hour < end_time.hour)]
            self.df.loc[mask.index, 'curfew'] = 1
            
        
        ###### Holidays variable ######

        # Consider only metropolitan areas
        holidays = holidays[holidays["Zones"].isin(["Zone C"])]

        # Consider only relevant years
        holidays = holidays[holidays["annee_scolaire"].isin(["2020-2021", "2021-2022"])]

        # Distinguish for holidays in Paris or not
        holidays['Holidays in Paris'] = 1

        holidays.drop(["Académies","Population", "Zones"], axis=1, inplace = True)

        # Convert to same date format
        holidays['Date de début'] = holidays['Date de début'].apply(lambda x: x[0:10]+' '+x[11:19])
        holidays['Date de fin'] = holidays['Date de fin'].apply(lambda x: x[0:10]+' '+x[11:19])

        holidays["Date de début"] = pd.to_datetime(holidays["Date de début"], format='%Y-%m-%d %H:%M:%S')
        holidays["Date de fin"] = pd.to_datetime(holidays["Date de fin"], format='%Y-%m-%d %H:%M:%S')

        # Remove holidays starting after final date of dataset
        holidays = holidays[holidays["Date de début"].dt.year != 2022]

        # Remove summer holidays
        holidays = holidays[holidays["Description"] != "Vacances d'Été"]
        holidays.drop(["Description","annee_scolaire"], axis=1, inplace=True)

        # Drop same holidays
        holidays = holidays.drop_duplicates(subset=['Date de début', 'Date de fin'], keep='first')

        # Create holidays date ranges 
        ranges = []
        for x in holidays[["Date de début","Date de fin"]].values:
            ranges.append(pd.date_range(x[0], x[1], freq="h"))
            
            
        def is_date_within_ranges(date, ranges):
            for date_range in ranges:
                if date_range[0] <= date <= date_range[-1]:
                    return 1
            return 0

        # Apply the function to create a new column indicating whether the date is within any holiday range
        self.df['holiday'] = self.df['date'].apply(lambda x: is_date_within_ranges(x, ranges))
        
        
        ###### Rush hour ########
        def is_rush_hour(x):
            if ((7 <= x.hour <= 9) or (16 <= x.hour <= 19)) and x.weekday != 5 and x.weekday != 6:
                return 1
            elif (x.weekday == 5 or x.weekday == 6) and (16 <= x.hour <= 19):
                return 1
            else:
                return 0
            
        self.df['rush_hour'] = self.df['date'].apply(lambda x: is_rush_hour(x))
        

    def output(self, test: bool=False):

        if test == False:

            # Using interpolation to fill in missing values before adding lag features
            linear_int = self.df[["date", "valeur_NO2", "valeur_CO", "valeur_O3", "valeur_PM10", "valeur_PM25"]]
            linear_int = linear_int.set_index("date")
            new_features = linear_int.resample('1h').interpolate("linear")

            self.df = self.df.merge(new_features, on="date", how="left")
            self.df = self.df.rename(columns={'valeur_NO2_y': 'valeur_NO2', 'valeur_CO_y': 'valeur_CO', 'valeur_O3_y': 'valeur_O3', 'valeur_PM10_y': 'valeur_PM10',
                'valeur_PM25_y': 'valeur_PM25'})
            self.df = self.df.drop(['valeur_NO2_x', 'valeur_CO_x', 'valeur_O3_x', 'valeur_PM10_x',
                'valeur_PM25_x'], axis=1)
            
            # Generating lag columns
            self.df["valeur_NO2_lag1"] = self.df['valeur_NO2'].shift(1)
            self.df["valeur_CO_lag1"] = self.df['valeur_CO'].shift(1)
            self.df["valeur_O3_lag1"] = self.df['valeur_O3'].shift(1)
            self.df["valeur_PM10_lag1"] = self.df['valeur_PM10'].shift(1)
            self.df["valeur_PM25_lag1"] = self.df['valeur_PM25'].shift(1)

            self.df["valeur_NO2_lag12"] = self.df['valeur_NO2'].shift(12)
            self.df["valeur_CO_lag12"] = self.df['valeur_CO'].shift(12)
            self.df["valeur_O3_lag12"] = self.df['valeur_O3'].shift(12)
            self.df["valeur_PM10_lag12"] = self.df['valeur_PM10'].shift(12)
            self.df["valeur_PM25_lag12"] = self.df['valeur_PM25'].shift(12)

            self.df["valeur_NO2_lag24"] = self.df['valeur_NO2'].shift(24)
            self.df["valeur_CO_lag24"] = self.df['valeur_CO'].shift(24)
            self.df["valeur_O3_lag24"] = self.df['valeur_O3'].shift(24)
            self.df["valeur_PM10_lag24"] = self.df['valeur_PM10'].shift(24)
            self.df["valeur_PM25_lag24"] = self.df['valeur_PM25'].shift(24)

            # Filling NaNs
            self.df['valeur_NO2_lag1'] = self.df['valeur_NO2_lag1'].fillna(self.df['valeur_NO2'])
            self.df['valeur_CO_lag1'] = self.df['valeur_CO_lag1'].fillna(self.df['valeur_CO'])
            self.df['valeur_O3_lag1'] = self.df['valeur_O3_lag1'].fillna(self.df['valeur_O3'])
            self.df['valeur_PM10_lag1'] = self.df['valeur_PM10_lag1'].fillna(self.df['valeur_PM10'])
            self.df['valeur_PM25_lag1'] = self.df['valeur_PM25_lag1'].fillna(self.df['valeur_PM25'])

            self.df['valeur_NO2_lag12'] = self.df['valeur_NO2_lag12'].fillna(self.df['valeur_NO2'])
            self.df['valeur_CO_lag12'] = self.df['valeur_CO_lag12'].fillna(self.df['valeur_CO'])
            self.df['valeur_O3_lag12'] = self.df['valeur_O3_lag12'].fillna(self.df['valeur_O3'])
            self.df['valeur_PM10_lag12'] = self.df['valeur_PM10_lag12'].fillna(self.df['valeur_PM10'])
            self.df['valeur_PM25_lag12'] = self.df['valeur_PM25_lag12'].fillna(self.df['valeur_PM25'])

            self.df['valeur_NO2_lag24'] = self.df['valeur_NO2_lag24'].fillna(self.df['valeur_NO2'])
            self.df['valeur_CO_lag24'] = self.df['valeur_CO_lag24'].fillna(self.df['valeur_CO'])
            self.df['valeur_O3_lag24'] = self.df['valeur_O3_lag24'].fillna(self.df['valeur_O3'])
            self.df['valeur_PM10_lag24'] = self.df['valeur_PM10_lag24'].fillna(self.df['valeur_PM10'])
            self.df['valeur_PM25_lag24'] = self.df['valeur_PM25_lag24'].fillna(self.df['valeur_PM25'])

            self.df.set_index("date", inplace=True)
        
        else:
            self.df.set_index("date", inplace=True)

        return self.df

### Preprocessing Class

In [26]:
class Preprocess:
    def __init__(self, df, test: bool=False):
        self.dfs = []
        targets = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']
        for col in targets:
            if test == False:
                self.df = df
                self.X = self.df.drop(targets, axis=1) 
                for col2 in targets:
                    if col2 == col:
                        continue
                    else:
                        self.X = self.X.drop([f'{col2}_lag1', f'{col2}_lag12', f'{col2}_lag24'], axis=1)
                y = self.df[col]

                self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, y, test_size=0.12, random_state=42, shuffle=False)

                self.pipe(test=False, col=col)
                self.dfs.append([self.data_train_prepared, self.data_test_prepared, self.y_train, self.y_test])
        
            else:
                self.df = df
                self.pipe(test=True, col=col)
                self.dfs.append(self.data_pred)
        
        
    def pipe(self, col: str, test: bool=False):
        
        categorical_features = ['year', 'holiday', 'rush_hour', 'lockdown', 'curfew']

        numerical_features = ['weekend', f'{col}_lag1', f'{col}_lag12', f'{col}_lag24', 'flow',
            'occupation_rate', 'ALTI', 'RR1', 'QRR1', 'T', 'QT', 'TN', 'QTN', 'QHTN', 'TX', 'QTX',
            'QHTX', 'DG', 'QDG']

        dates = ["weekday", "month", "hour", 'day']

        cyclical = CyclicalFeatures(variables=None, drop_original=True)

        full_pipeline = ColumnTransformer([
                ("cat", OneHotEncoder(sparse_output=False), categorical_features),
                ("num", MinMaxScaler(), numerical_features), 
                ("dates", cyclical, dates),
            ], remainder='passthrough').set_output(transform='pandas')
        
        if test == False:
            self.data_train_prepared = full_pipeline.fit_transform(self.X_train)
            self.data_test_prepared = full_pipeline.transform(self.X_test)

        else:
            self.data_pred = full_pipeline.fit_transform(self.df)
            self.data_pred['cat__curfew_1'] = 0
            self.data_pred['cat__holiday_1'] = 0
            self.data_pred['cat__lockdown_1'] = 0
            self.data_pred['cat__year_2020'] = 0
            self.data_pred['cat__year_2021'] = 0
            self.data_pred['cat__year_2022'] = 0
            self.data_pred['cat__year_2023'] = 0

## Preprocessing

In [27]:
eng = Eng(df, weather, traffic, holidays)
df = eng.output(test=False)
eng1 = Eng(final_test, weather, traffic, holidays)
df_test = eng1.output(test=True)

# Transforming values with log function to decrease influence of outliers
for col in ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']:
    df[col].apply(lambda x: np.log(x) if x > 0 else 0)

# Shifting the lags by one position to initialize predictions
lags = [1, 12, 24]
for lag in lags:
    df_test[f'valeur_NO2_lag{lag}'] = df[f'valeur_NO2_lag{lag}'].iloc[-1]
    df_test[f'valeur_CO_lag{lag}'] = df[f'valeur_CO_lag{lag}'].iloc[-1]
    df_test[f'valeur_O3_lag{lag}'] = df[f'valeur_O3_lag{lag}'].iloc[-1]
    df_test[f'valeur_PM10_lag{lag}'] = df[f'valeur_PM10_lag{lag}'].iloc[-1]
    df_test[f'valeur_PM25_lag{lag}'] = df[f'valeur_PM25_lag{lag}'].iloc[-1]

data = Preprocess(df, test=False)
data_test = Preprocess(df_test, test=True)

In [ ]:
dfs_train = data.dfs
dfs_test = data_test.dfs

## Using recursive forecasting to update lags as we go

In [8]:
def recursive_forecast(df: pd.DataFrame, model, target_col: str, forecast_steps=504):

    # Initialize predictions dictionary
    predictions = []  # Store predictions

    # Get a list of indices to ensure sequential row access by index
    indices = df.index.tolist()
    
    for step in range(forecast_steps):
        # Get the current row based on the index
        current_index = indices[step]
        latest_row = df.loc[current_index].copy()  # Copy to avoid modifying df directly
        
        # Prepare current input by selecting lagged features and other relevant columns
        X_current = pd.DataFrame([latest_row])  # Single-row DataFrame with current features

        if target_col == "valeur_NO2":
            X_current[[
        f'num__{target_col}_lag1',
        f'num__{target_col}_lag12', 
        f'num__{target_col}_lag24',
    ]] = (X_current[[
        f'num__{target_col}_lag1',
        f'num__{target_col}_lag12', 
        f'num__{target_col}_lag24',
    ]] - 1.9)/ (131 - 1.9)
        
        elif target_col == "valeur_CO":
            X_current[[
        f'num__{target_col}_lag1',
        f'num__{target_col}_lag12', 
        f'num__{target_col}_lag24',
    ]] =  (X_current[[
        f'num__{target_col}_lag1',
        f'num__{target_col}_lag12', 
        f'num__{target_col}_lag24',
    ]] - 0.037)/ (4.309 - 0.037)
        
        elif target_col == "valeur_O3":
            X_current[[
        f'num__{target_col}_lag1',
        f'num__{target_col}_lag12', 
        f'num__{target_col}_lag24',
    ]] /= 193
        
        elif target_col == "valeur_PM10":
            X_current[[
        f'num__{target_col}_lag1',
        f'num__{target_col}_lag12', 
        f'num__{target_col}_lag24',
    ]] = (X_current[[
        f'num__{target_col}_lag1',
        f'num__{target_col}_lag12', 
        f'num__{target_col}_lag24',
    ]] - 0.5)/ 128
        
        else:
            X_current[[
        f'num__{target_col}_lag1',
        f'num__{target_col}_lag12', 
        f'num__{target_col}_lag24',
    ]] /= 111.1
    
        # Predict all target variables at once
        forecast_values = model.predict(X_current)[0] # Assuming model outputs a single array of predictions

        # Store the predictions for this step
        predictions.append(forecast_values)

        # Update lagged values in the next row if it exists
        if step < len(indices) - 1:  # Check if a next row exists
            next_index = indices[step + 1]  # Find the next date index
            index = indices[step]
            # Shift lagged values backwards in the next row
            
            df.at[next_index, f'num__{target_col}_lag12'] = df.at[index, f'num__{target_col}_lag1']
            df.at[next_index, f'num__{target_col}_lag24'] = df.at[index, f'num__{target_col}_lag12']
            # Set the latest forecast as lag1 in the next row
            df.at[next_index, f'num__{target_col}_lag1'] = forecast_values

    return predictions, df

## Hyper-parameter tuning for Catboost

In [9]:
############# NO NEED TO RUN AGAIN #############

param_grid = {
    'learning_rate': [0.05, 0.1, 0.15],
    'depth': [4, 6, 8],
    'iterations': [500, 1250, 2000],
    'l2_leaf_reg': [1, 3, 5],
    'bagging_temperature': [0.5, 1.0, 1.5],
    'border_count': [32, 64, 128],  # Number of splits for numerical features
    'subsample': [0.8, 1.0],  # Fraction of samples used for training each tree
    'colsample_bylevel': [0.8, 1.0],  # Fraction of features used for training each level
    'min_child_samples': [1, 5, 10]  # Minimum number of samples in each leaf
}

best_params = []

for i in dfs_train:
    data_train_prepared  = i[0].drop(i[0].filter(regex='^remainder').columns, axis=1)
    y_train = i[2]

    catboost = CatBoostRegressor(verbose=False)

    random_search = RandomizedSearchCV(
        catboost,
        param_distributions=param_grid,
        n_iter=10,  # Number of random combinations to try
        scoring='neg_root_mean_squared_error',
        cv=TimeSeriesSplit(n_splits=6).split(data_train_prepared, y_train),  # Number of cross-validation folds
    )

    # Fit the RandomizedSearchCV object to the data
    random_search.fit(data_train_prepared, y_train)  

    # Print the best parameters and corresponding score
    print("Best Parameters: ", random_search.best_params_)
    print("Best Score: ", random_search.best_score_)

    best_params.append(random_search.best_params_)

############# NO NEED TO RUN AGAIN #############

Best Parameters:  {'subsample': 1.0, 'min_child_samples': 10, 'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 500, 'depth': 8, 'colsample_bylevel': 0.8, 'border_count': 128, 'bagging_temperature': 1.0}
Best Score:  -5.16041218629461
Best Parameters:  {'subsample': 0.8, 'min_child_samples': 1, 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 500, 'depth': 6, 'colsample_bylevel': 0.8, 'border_count': 128, 'bagging_temperature': 1.0}
Best Score:  -0.04104316596408841
Best Parameters:  {'subsample': 1.0, 'min_child_samples': 5, 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 500, 'depth': 4, 'colsample_bylevel': 0.8, 'border_count': 64, 'bagging_temperature': 0.5}
Best Score:  -6.382551693713932
Best Parameters:  {'subsample': 0.8, 'min_child_samples': 1, 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 500, 'depth': 4, 'colsample_bylevel': 1.0, 'border_count': 128, 'bagging_temperature': 1.5}
Best Score:  -3.912205573175988
Best Parameters:  {'subsample': 1.0, 'm

In [15]:
# Use these hyper-parameters for fitting the Catboost regressors to each target variable
print(f'Use {best_params[0]} for valeur_NO2', "\n",
      f'Use {best_params[1]} for valeur_CO', "\n",
      f'Use {best_params[2]} for valeur_O3', "\n",
      f'Use {best_params[3]} for valeur_PM10', "\n",
      f'Use {best_params[4]} for valeur_PM25', "\n")

Use {'subsample': 1.0, 'min_child_samples': 10, 'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 500, 'depth': 8, 'colsample_bylevel': 0.8, 'border_count': 128, 'bagging_temperature': 1.0} for valeur_NO2 
 Use {'subsample': 0.8, 'min_child_samples': 1, 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 500, 'depth': 6, 'colsample_bylevel': 0.8, 'border_count': 128, 'bagging_temperature': 1.0} for valeur_CO 
 Use {'subsample': 1.0, 'min_child_samples': 5, 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 500, 'depth': 4, 'colsample_bylevel': 0.8, 'border_count': 64, 'bagging_temperature': 0.5} for valeur_O3 
 Use {'subsample': 0.8, 'min_child_samples': 1, 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 500, 'depth': 4, 'colsample_bylevel': 1.0, 'border_count': 128, 'bagging_temperature': 1.5} for valeur_PM10 
 Use {'subsample': 1.0, 'min_child_samples': 1, 'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 1250, 'depth': 6, 'colsample_bylevel': 1.0, 'border_cou

## Predictions

In [10]:
final_test = pd.read_csv("../data/test.csv", index_col=0, parse_dates=[0])
targets = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']

for i, j, k, l in zip(dfs_train, dfs_test, targets, best_params):
    data_train_prepared  = i[0].drop(i[0].filter(regex='^remainder').columns, axis=1)
    y_train = i[2]

    data_test_prepared = i[1].drop(i[1].filter(regex='^remainder').columns, axis=1)
    y_test = i[3]

    data_pred = j.drop(j.filter(regex='^remainder').columns, axis=1)

    # Define the specific index where the first row of `final_test_processed` is located
    first_index = data_pred.index[0]  # Replace with the actual first index if known

    # Extract the last three rows of `y_test`
    last_three_rows = y_test[-3:]

    # Assign lagged values to the specified row of `final_test_processed`
    data_pred.at[first_index, f'num__{k}_lag24'] = last_three_rows.iloc[0]  # 3rd last row as lag3
    data_pred.at[first_index, f'num__{k}_lag12'] = last_three_rows.iloc[1]  # 2nd last row as lag2
    data_pred.at[first_index, f'num__{k}_lag1'] = last_three_rows.iloc[2]  # Last row as lag1
        
    forest = CatBoostRegressor(**l, verbose=False)
    
    miaou = forest.fit(data_train_prepared, y_train)

    predictions, final_df = recursive_forecast(data_pred, miaou, target_col=k, forecast_steps=504)

    df_submit = pd.DataFrame(predictions)

    final_test = pd.concat([final_test, df_submit], axis=1)

final_test = final_test[504:]
final_test.columns = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']

C:\Users\jules\AppData\Local\Temp\ipykernel_13740\3671204366.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  final_test = pd.concat([final_test, df_submit], axis=1)


In [11]:
test = pd.read_csv("../data/test.csv", index_col=0)
final_test.index = test.index

# Making sure no prediction is negative as it makes no sense
for col in final_test.columns:
    final_test[col].apply(lambda x: 0 if x < 0 else x)

In [12]:
final_test.to_csv("../output/submission.csv")

## Testing against the test set

In [13]:
y_test = pd.concat([dfs_train[0][3], dfs_train[1][3], dfs_train[2][3], dfs_train[3][3], dfs_train[4][3]], axis=1)
y_test.columns = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']

In [14]:
from sklearn.metrics import mean_absolute_error
import numpy as np

def calculate_average_mae(predictions, actuals, target_columns):
    """
    Calculate the MAE for each pollutant and return the average MAE.

    Parameters:
    - predictions: dict of lists where keys are target columns and values are lists of predicted values
    - actuals: DataFrame containing the actual values of the pollutants
    - target_columns: List of target column names corresponding to pollutants
    
    Returns:
    - average_mae: float, average MAE across the pollutants
    """
    mae_values = []
    
    for target in target_columns:
        # Calculate MAE for each pollutant
        mae = mean_absolute_error(actuals[target], predictions[target])
        mae_values.append(mae)
        print(F"mae for {target} is {mae}")
    
    # Calculate the average MAE
    average_mae = np.mean(mae_values)
    return average_mae

# Example usage
target_columns = ["valeur_NO2", "valeur_CO", "valeur_O3", "valeur_PM10", "valeur_PM25"]
average_mae = calculate_average_mae(final_test, y_test[len(y_test)-504:], target_columns)
print("Average MAE:", average_mae)

mae for valeur_NO2 is 15.353618356083375
mae for valeur_CO is 0.3345154653977376
mae for valeur_O3 is 17.439246958140885
mae for valeur_PM10 is 7.035681925923305
mae for valeur_PM25 is 4.464356065545884
Average MAE: 8.925483754218238
